In [ ]:
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
from keras.utils.visualize_util import plot
from keras.models import model_from_json



%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sys
import time
import png

nb_epoch = 10

In [ ]:
#Load origional MNIST Data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
#seperate zeros from the rest of the data
zeros_seperate_images = []
for i in range(len(y_train)):
    if y_train[i] == 9:
        zeros_seperate_images.append(X_train[i])
        

In [ ]:
#Add noise to the images

noise = 50 #Must be a number between 0 and 255

def addNoiseToImage(X, noiseRange):
    for i in range(len(X)):
        for j in range(len(X[0])):
            if X[i][j] == 0:
                X[i][j] = np.random.random()*noiseRange
            

In [ ]:
# Function for saving output to file for black and
# white viewing and future reference
# takes in a np.array and writes it to a png file
def saveImage(s,fileName):
    f = open(fileName, 'wb')      # binary mode is important
    w = png.Writer(len(s[0]), len(s), greyscale=True, bitdepth=8)
    w.write(f, s)
    f.close()


In [ ]:
addNoiseToImage(zeros_seperate_images[1], noise)

In [ ]:
print('Number of Zeros: ', len(zeros_seperate_images))
plt.imshow(zeros_seperate_images[9])
plt.title('zeros1seperate_images example')
plt.show()

In [ ]:
#Concatonation Function

# A function that takes in a 
# np.array(size:numberofImages x 28 x 28)
# mnist pictures in the first parameter and 
# the desired ouput size of one side of 
# the square final picture in the second parameter 
# and creates an imagewith the number placed 
# randomly in the image without overlapping

#the top left number in the image is the number of
#objects in the image, just did this for convenience

def insertImage(A, B, topLeftCoordinate):
    xi = topLeftCoordinate[0]
    yi = topLeftCoordinate[1]
    for i in range(28):
        for j in range(28):
            A[xi+i][yi+j] = B[i][j]

def updatePlacementMatrix(A, topLeftCoordinate):
    xi = topLeftCoordinate[0]
    yi = topLeftCoordinate[1]
    for i in range(28):
        for j in range(28):
            A[xi+i][yi+j] = 0
    

def concatonatedMnistData(images, size):
    newImg = [i[:] for i in [[0]*size]*size] #makes new matrix of zeros
    
    #a is a matrix that uses binary to keep track of inserted
    a = [i[:] for i in [[1]*size]*size] 
    
    placedNumbers = 0 
    iterations = 0
    while (iterations < 100) and (placedNumbers < len(images)):
        x = int(size*np.random.random()%(size-28))
        y = int(size*np.random.random()%(size-28))
        if a[x][y] and a[x][y+28] and a[x+28][y] and a[x+28][y+28]:
            updatePlacementMatrix(a, [x,y])
            insertImage(newImg, images[placedNumbers],[x,y])
            placedNumbers+=1
            iterations = 0
        iterations+=1
    newImg[0][0] = placedNumbers 
    return np.array(newImg),a
    

In [ ]:
inputX,outputY = concatonatedMnistData(zeros_seperate_images[3:100], 100)
number_of_placed = inputX[0][0]
print number_of_placed

In [ ]:
plt.imshow(outputY)
plt.title('Resulting Box Data')
plt.show()

In [ ]:
plt.imshow(inputX)
plt.title('Resulting Concatonated Image Data')
plt.show()

In [ ]:
# Lets make some noise!!!!!
noise = 50
addNoiseToImage(inputX, noise)

In [ ]:
plt.imshow(inputX)
plt.title('Resulting noisy data example')
plt.show()

In [ ]:
saveImage(inputX, 'example.png')

In [ ]:
def makeConcatonatedImageData(images, newSize, maxImagesInNewImage):
    unplacedImages = len(images)-1
    i = 0
    j = maxImagesInNewImage
    Xinput = []
    Youtput = []
    Ycount = []
    while (unplacedImages > 0):
        x,y = concatonatedMnistData(images[i:j], newSize)
        number_of_placed = x[0][0]
        Xinput.append(x)
        Youtput.append(y)
        Ycount.append(number_of_placed)
        i += number_of_placed
        j += number_of_placed
        unplacedImages -= number_of_placed
    return Xinput, Youtput, Ycount
        
        
    

In [ ]:
X, Y1, Y2 = makeConcatonatedImageData(zeros_seperate_images, 100, 4)
print len(X)

In [ ]:
plt.imshow(X[1400])

In [ ]:
X_test = np.array(X[1400:])
y_test = np.array(Y1[1400:])
Y2test = np.array(Y2[1400:])
X_train = np.array(X[:1400])
y_train = np.array(Y1[:1400])
Y2 = np.array(Y2[:1400])

In [ ]:
num_train = X_train.shape[0]
num_test = X_test.shape[0]
im_width  = X_train.shape[1]
im_height = X_train.shape[2]
print('shape before')
print(X_train.shape)

#resize data for use in convolutional network
X_train = X_train.reshape(num_train,1, im_width,im_height)
X_test = X_test.reshape(num_test,1, im_width,im_height)

print('shape after')
print(X_train.shape)

In [ ]:
# change type to float
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# normalize the range
print('maximum of X_train:',np.max(X_train[:]))
X_train /= 255.0;
X_test /= 255.0;
print('maximum of X_train:',np.max(X_train[:]))

# convert class vectors to binary class matrices (one hot representation)
nb_classes = np.unique(y_train).size
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [ ]:
# construct the network
model = Sequential()
model.add(Convolution2D(64, 10, 10, border_mode='same', input_shape=(1,im_width,im_height)))
model.add(Activation('relu'))
model.add(Dropout(0.23))
model.add(Convolution2D(16, 2, 2, border_mode='same', input_shape=(1,im_width,im_height)))
model.add(Activation('relu'))
model.add(Dropout(0.23))
model.add(MaxPooling2D(pool_size=(2,2), strides=None, border_mode='valid'))
model.add(Flatten())
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.23))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [ ]:
batch_size = 1024
start = time.time()
history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_test, Y_test))
end = time.time()